In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, t
import riskmgmlib as rml

I used Xiaokuan Zhao

My netid is xz358

This is my hashed name:  bdc6aa95f87584754400b13561033edef6b044cf

In [54]:
# Problem1
# Using the data in “problem1.csv”
# a. Calculate Log Returns (2pts)
# b. Calculate Pairwise Covariance (4pt)
# c. Is this Matrix PSD? If not, fix it with the “near_psd” method (2pt)
# d. Discuss when you might see data like this in the real world. (2pt)

data1 = pd.read_csv('problem1.csv',index_col=3)

#a
logreturndata1 = np.log(data1/data1.shift(1)).dropna()
print(logreturndata1)
print("")

#b

data1_12 = data1[(~data1['Price1'].isna()) & (~data1['Price2'].isna())]
var1 = np.var(~data1['Price1'].isna())
cov12 = np.cov(data1_12['Price1'], data1_12['Price2'])[0][1]

data1_13 = data1[(~data1['Price1'].isna()) & (~data1['Price3'].isna())]
var3 = np.var(~data1['Price3'].isna())
cov13 = np.cov(data1_13['Price1'], data1_13['Price3'])[0][1]

data1_23 = data1[(~data1['Price2'].isna()) & (~data1['Price3'].isna())]
var2 = np.var(~data1['Price2'].isna())
cov_23 = np.cov(data1_23['Price2'], data1_23['Price3'])[0][1]

cov_matrix = [[var1,cov12,cov13],[cov12,var2,cov_23],[cov13,cov_23,var3]]
print('Covariance Matrix: \n',cov_matrix)
print("")
#c
is_psd = rml.is_psd(cov_matrix)
print('Is this Matrix PSD? ',is_psd)
print("")
fixed_matrix = rml.near_psd(cov_matrix)
print('Fixed Matrix: \n',fixed_matrix)

#d
# Data like this can be seen in the real world when the data is collected from different exchanges since
# different exchanges have different different holidays. So the data is not complete.



              Price1    Price2    Price3
Date                                    
2023-04-13  0.018993  0.003195 -0.000451
2023-04-14  0.002168  0.007144  0.002393
2023-04-15 -0.006809 -0.002536 -0.000854
2023-04-16 -0.003761 -0.004983 -0.002282
2023-04-19 -0.001601 -0.005461 -0.001421
2023-04-20  0.002864  0.000578  0.001313
2023-04-21 -0.001993  0.003972 -0.000706
2023-04-24  0.003410  0.005293 -0.000427
2023-04-25  0.006229  0.003376  0.000830
2023-04-26 -0.006950 -0.005061 -0.000667
2023-04-27  0.009240  0.004083  0.001317
2023-04-28  0.008836 -0.002461 -0.000957

Covariance Matrix: 
 [[0.09000000000000001, 0.11616778283337, 0.01289133815358598], [0.11616778283337, 0.09000000000000001, 0.015150806409572592], [0.01289133815358598, 0.015150806409572592, 0.09]]

Is this Matrix PSD?  False

Fixed Matrix: 
 [[0.09       0.08998502 0.01228759]
 [0.08998502 0.09       0.01391191]
 [0.01228759 0.01391191 0.09      ]]


In [10]:
# “problem2.csv” contains data about a call option. Time to maturity is given in days. Assume 255
# days in a year.
# a. Calculate the call price (1pt)
# b. Calculate Delta (1pt)
# c. Calculate Gamma (1pt)
# d. Calculate Vega (1pt)
# e. Calculate Rho (1pt)
# Assume you are long 1 share of underlying and are short 1 call option. Using Monte Carlo
# assuming a Normal distribution of arithmetic returns where the implied volatility is the annual
# volatility and 0 mean
# f. Calculate VaR at 5% (2pt)
# g. Calculate ES at 5% (2pt)
# h. This portfolio’s payoff structure most closely resembles what? (1pt)

data2 = pd.read_csv('problem2.csv')
S = data2.Underlying.values[0]
X = data2.Strike.values[0]
sigma = data2.IV.values[0]
T = data2.TTM.values[0]/255
r = data2.RF.values[0]
div = data2.DivRate.values[0]
b = r - div

#a
price = rml.gbsm('C',S, X, r, b, sigma, T)
print('The call price is: ',price)
print("")
#b
delta_call = np.exp((b-r)*T)*norm.cdf(rml.d1(S, X, b, sigma, T))
print('The delta of the call is: ',delta_call)
print("")
#c
gamma = np.exp((b-r)*T)*norm.pdf(rml.d1(S, X, b, sigma, T))/(S*sigma*np.sqrt(T))
print('The gamma of the call is: ',gamma)
print("")
#d
vega = S*np.exp((b-r)*T)*norm.pdf(rml.d1(S, X, b, sigma, T))*np.sqrt(T)
print('The vega of the call option is: ', vega)
print("")
#e
rho_call = -T*S*np.exp(b*T - r*T)*norm.cdf(rml.d1(S, X, b, sigma, T)) + X*T*np.exp(-r*T)*norm.cdf(rml.d2(S, X, b, sigma, T))
print('Rho of the call option is: ', rho_call)
print("")
#f
T = (data2.TTM.values[0]-1)/255
# monte carlo simulation from S, assuming a Normal distribution of arithmetic returns where the implied volatility is 0.23 and 0 mean
underlying_price = S * (1+norm.rvs(loc=0, scale=sigma, size=10000))
pnl = underlying_price - S - [underlying_price for underlying_price in rml.gbsm('C', underlying_price, X, r, b, sigma ,T)] + price
var = rml.calculate_var(pnl, np.mean(pnl))
print('VaR at 5% is: ', var)
print("")
#g
es = rml.calculate_es(pnl, np.mean(pnl))
print('ES at 5% is: ', es)
print("")
#h
print('This is similar to a covered call')

The call price is:  6.801866322511721

The delta of the call is:  0.5084801613020203

The gamma of the call is:  0.023104378660025782

The vega of the call option is:  28.309901459443246

Rho of the call option is:  -3.6872810452777642

VaR at 5% is:  60.461567591870654

ES at 5% is:  79.06241228618667

This is similar to a covered call


In [18]:
# Problem 3
# Data in “problem3_cov.csv” is the covariance for 3 assets. “problem3_ER.csv” is the expected
# return for each asset as well as the risk free rate.
# a. Calculate the Maximum Sharpe Ratio Portfolio (4pt)
# b. Calculate the Risk Parity Portfolio (4pt)
# c. Compare the differences between the portfolio and explain why. (2pt)

cov_matrix = pd.read_csv('problem3_cov.csv')
cov_matrix = list(cov_matrix.values)
er = pd.read_csv('problem3_ER.csv')
rf = er.RF.values[0]
er.drop('RF',axis=1,inplace=True)
er = list(er.values[0])
volatility = np.sqrt([cov_matrix[0][0], cov_matrix[1][1], cov_matrix[2][2]])
corr = rml.corr(cov_matrix)

#a
wos, ratiio = rml.optimize_sharpe_ratio(er, volatility, corr)
print(wos)
print("")
#b
wrp = rml.risk_parity_weight(corr,volatility)
print(wrp)

[0.37140965 0.34794704 0.2806433 ]

[0.352  0.3394 0.3086]


In [5]:
print(f"ER Optimal: {wos.dot(er)}")
print(f"SD Optimal: {np.sqrt(wos.T.dot(cov_matrix).dot(wos))}")
print(f"SR Optimal: {(wos.T.dot(er) - rf) / np.sqrt(wos.T.dot(cov_matrix).dot(wos))}")
print(" ")
print(f"ER RP: {wrp.dot(er)}")
print(f"SD RP: {np.sqrt(wrp.T.dot(cov_matrix).dot(wrp))}")
print(f"SR RP: {(wrp.T.dot(er) - rf) / np.sqrt(wrp.T.dot(cov_matrix).dot(wrp))}")

ER Optimal: 0.14115218197083818
SD Optimal: 0.18035036046673555
SR Optimal: 0.5331410578942137
 
ER RP: 0.14148229019665012
SD RP: 0.18095388639326654
SR RP: 0.5331871678454336


In [6]:
print(corr)

[[1.   0.88 0.88]
 [0.88 1.   0.88]
 [0.88 0.88 1.  ]]


As we can see, the maximum sharpe ratio portfolio is [0.37, 0.35, 0.28], and the risk parity portfolio is [0.35, 0.34, 0.31]. There is difference but the difference is small. Also, their ER, SD and SR is similar too. This is because the correlation matrixes are equal, so the risk parity portfolio is the max sharpe ratio portfolio. The little difference is perhaps because of digital accuracy.

In [3]:
# 4. Data in “problem4_returns.csv” is a series of returns for 3 assets. “problem4_startWeight.csv” is
# the starting weights of a portfolio of these assets as of the first day in the return series.
# a. Calculate the new weights for the start of each time period (2pt)
# b. Calculate the ex-post return attribution of the portfolio on each asset (4pt)
# c. Calculate the ex-post risk attribution of the portfolio on each asset (2pt)

data4_returns = pd.read_csv('problem4_returns.csv', index_col=3)
data4_weights = pd.read_csv('problem4_startWeight.csv')
data4_weights = list(data4_weights.values[0])


stocks = ['Asset1', 'Asset2', 'Asset3']
attribution, weight = rml.ex_post_contribution(data4_weights, stocks, data4_returns)

#a
print(weight)
print("")
#b,c
print(attribution)


[[0.43226518 0.15549249 0.41224233]
 [0.44480495 0.15171745 0.4034776 ]
 [0.45716808 0.16549373 0.37733819]
 [0.46624717 0.16555259 0.36820024]
 [0.47921907 0.15739076 0.36339018]
 [0.46930371 0.16718065 0.36351564]
 [0.46969817 0.16270907 0.36759276]
 [0.42953883 0.17382824 0.39663293]
 [0.4201786  0.16851823 0.41130318]
 [0.40786885 0.16518321 0.42694794]
 [0.43319934 0.16759738 0.39920328]
 [0.41514963 0.15558505 0.42926532]
 [0.44915144 0.15009118 0.40075737]
 [0.45214956 0.15247169 0.39537876]
 [0.44972802 0.16321312 0.38705886]
 [0.43479535 0.15645844 0.40874621]
 [0.42310482 0.17341374 0.40348145]
 [0.4336153  0.17309293 0.39329177]
 [0.45661383 0.17155924 0.37182694]
 [0.43953408 0.16967044 0.39079548]]

     Asset1    Asset2    Asset3  Portfolio               Value
0 -0.024677  0.157992 -0.089881  -0.023153        Total Return
1 -0.012221  0.024968 -0.035901  -0.023153  Return Attribution
2  0.012564  0.000161  0.021275   0.033999     Vol Attribution


In [22]:
# Problem5
# Input prices in “problem5.csv” are for a portfolio. You hold 1 share of each asset. Using
# arithmetic returns, fit a generalized T distribution to each asset return series. Using a Gaussian
# Copula:
# a. Calculate VaR (5%) for each asset (3pt)
# b. Calculate VaR (5%) for a portfolio of Asset 1 &2 and a portfolio of Asset 3&4 (4pt)
# c. Calculate VaR (5%) for a portfolio of all 4 assets. (3pt)
data5 = pd.read_csv('problem5.csv', index_col=4)
returns = data5.pct_change().dropna()

In [23]:
def cal_t_pVals(port, returns_port, price):
    return_cdf=[]
    par=[]
    for col in returns_port.columns:
        df, loc, scale = t.fit(returns_port[col].values)
        par.append([df,loc,scale])
        return_cdf.append(t.cdf(returns_port[col].values, df=df, loc=loc, scale=scale).tolist())
    return_cdf=pd.DataFrame(return_cdf).T
    spearman_cor=return_cdf.corr(method='spearman')
    sample=pd.DataFrame(rml.PCA_with_percent(spearman_cor)).T
    sample_cdf=[]
    for col in sample.columns:
        sample_cdf.append(norm.cdf(sample[col].values, loc=0, scale=1).tolist())
    simu_return=[]
    for i in range(len(sample_cdf)):
        simu_return.append(t.ppf(sample_cdf[i], df=par[i][0], loc=par[i][1], scale=par[i][2]))
    simu_return=np.array(simu_return)

    sim_price=(1 + simu_return.T)*price
    pVals = sim_price.dot(port['Holding'])
    pVals.sort()
    return pVals

In [43]:
#a
port_1 = pd.DataFrame({'Holding':[1,0,0,0]},index=data5.columns)
price1 = [data5.iloc[-1].Price1,0,0,0]
pVals_price1 = cal_t_pVals(port_1, pd.DataFrame(returns['Price1']), price1)
print("Price1:")
print("VaR: ", rml.calculate_var(pVals_price1, data5.iloc[-1].Price1))
print("ES: ", rml.calculate_es(data5.iloc[-1].Price1 - pVals_price1))
print("")
port_2 = pd.DataFrame({'Holding':[0,1,0,0]},index=data5.columns)
price2 = [0,data5.iloc[-1].Price2,0,0]
pVals_price2 = cal_t_pVals(port_2, pd.DataFrame(returns['Price2']), price2)
print("Price2:")
print("VaR: ", rml.calculate_var(pVals_price2, data5.iloc[-1].Price2))
print("ES: ", rml.calculate_es(data5.iloc[-1].Price2 - pVals_price2))
print("")
port_3 = pd.DataFrame({'Holding':[0,0,1,0]},index=data5.columns)
price3 = [0,0,data5.iloc[-1].Price3,0]
pVals_price3 = cal_t_pVals(port_3, pd.DataFrame(returns['Price3']), price3)
print("Price3:")
print("VaR: ", rml.calculate_var(pVals_price3, data5.iloc[-1].Price3))
print("ES: ", rml.calculate_es(data5.iloc[-1].Price3 - pVals_price3))
print("")
port_4 = pd.DataFrame({'Holding':[0,0,0,1]},index=data5.columns)
price4 = [0,0,0,data5.iloc[-1].Price4]
pVals_price4 = cal_t_pVals(port_4, pd.DataFrame(returns['Price4']), price4)
print("Price4:")
print("VaR: ", rml.calculate_var(pVals_price4, data5.iloc[-1].Price4))
print("ES: ", rml.calculate_es(data5.iloc[-1].Price4 - pVals_price4))
print("")

Price1:
VaR:  0.04891275687040775
ES:  0.10489991622767447

Price2:
VaR:  0.09871574198213295
ES:  0.19094033270882227

Price3:
VaR:  0.09403475703427944
ES:  0.1933242642515971

Price4:
VaR:  0.07867808610927796
ES:  0.14823893554428932



In [52]:
#b
port_5 = pd.DataFrame({'Holding':[1,1]},index=['Price1', 'Price2'])
price5 = [data5.iloc[-1].Price1,data5.iloc[-1].Price2]
pVals_price5 = cal_t_pVals(port_5, pd.DataFrame(returns[['Price1','Price2']]), price5)
print("Price1 and Price2:")
print("VaR: ", rml.calculate_var(pVals_price5, data5.iloc[-1].Price1 + data5.iloc[-1].Price2))
print("ES: ", rml.calculate_es(data5.iloc[-1].Price1 + data5.iloc[-1].Price2 - pVals_price5))
print("")

port_6 = pd.DataFrame({'Holding':[1,1]},index=["Price3", "Price4"])
price6 = [data5.iloc[-1].Price3,data5.iloc[-1].Price4]
pVals_price6 = cal_t_pVals(port_6, pd.DataFrame(returns[['Price3','Price4']]), price6)
print("Price3 and Price4:")
print("VaR: ", rml.calculate_var(pVals_price6, data5.iloc[-1].Price3 + data5.iloc[-1].Price4))
print("ES: ", rml.calculate_es(data5.iloc[-1].Price3 + data5.iloc[-1].Price4 - pVals_price6))
print("")

Price1 and Price2:
VaR:  0.1265274558131182
ES:  0.2515723412821313

Price3 and Price4:
VaR:  0.14317437418844747
ES:  0.27835547661766824



In [53]:
#c
port_7 = pd.DataFrame({'Holding':[1,1,1,1]},index=data5.columns)
price7 = [data5.iloc[-1].Price1,data5.iloc[-1].Price2,data5.iloc[-1].Price3,data5.iloc[-1].Price4]
pVals_price7 = cal_t_pVals(port_7, pd.DataFrame(returns), price7)
print("Price1, Price2, Price3 and Price4:")
print("VaR: ", rml.calculate_var(pVals_price7, data5.iloc[-1].Price1 + data5.iloc[-1].Price2 + data5.iloc[-1].Price3 + data5.iloc[-1].Price4))
print("ES: ", rml.calculate_es(data5.iloc[-1].Price1 + data5.iloc[-1].Price2 + data5.iloc[-1].Price3 + data5.iloc[-1].Price4 - pVals_price7))
print("")

Price1, Price2, Price3 and Price4:
VaR:  0.2228997256864318
ES:  0.402755071498205

